# Entanglement Entropy

Here we explore how to calculate entropy using python.
First lets see what are the things we have learned from chapter 11 Nielsen's book of quantum computation, and Vinay's lecture notes.

## Pure and mixed states

When we know the exact state of a quantum system, then we can say that the system is in a pure state. The density matrix of such system can be written as,

$$
\hat{\rho} = \ket{\psi}\bra{\psi}.
$$
The probability of getting this state is then 1, since we know exactly that the system are occupying the state $\ket{\psi}$. If we have no idea what state the quantum system currently occupy then the state is in a mixed state condition. Mixed states can be written as,

$$
\hat{\rho} = \sum_i p_i \ket{\psi_i}\bra{\psi_i},
$$
where index "$i$" indicates the states that are possible for the system to be in. Example : A state of spin-1/2, and we say that the system have a probability 50% of being in an up or down state,

$$
\hat{\rho} = \frac{1}{2}\ket{\psi_1}\bra{\psi_i} + \frac{1}{2}\ket{\psi_2}\bra{\psi_2},
$$
with $\ket{\psi_1} = \frac{1}{\sqrt{2}}\ket{0} + \frac{1}{\sqrt{2}}\ket{1} $ and $\ket{\psi_2} = \ket{1}$ as the two possible states the system could occupy.

First we import the necessary libraries in order to calculate the entropy

In [3]:
import numpy as np
import matplotlib as mpl
import scipy as scp

In [4]:
from scipy.linalg import logm

# Define the density matrix of the quantum spin chain
Density_Matrix = np.array([[0.4, 0.2], [0.2, 0.6]])

# Compute the eigenvalues of the density matrix
Eigenvalues = np.linalg.eigvalsh(Density_Matrix)

# Filter out eigenvalues close to zero
Filtered_Eigenvalues = Eigenvalues[np.abs(Eigenvalues) > 1e-12]

# Calculate the logarithm of the filtered eigenvalues
Log_Eigenvalues = np.log(Filtered_Eigenvalues)

# Compute the entanglement entropy using the von Neumann entropy formula
Entanglement_Entropy = -np.sum(Filtered_Eigenvalues * Log_Eigenvalues)

print("Entanglement Entropy:", Entanglement_Entropy)

Entanglement Entropy: 0.5895144857350482


Von-neumann entropy

In [5]:
import numpy as np
from scipy.linalg import logm

# Define the reduced density matrix
rho_A = np.array([[0.7, 0.6], [0.3, 0.4]])

# Compute the eigenvalues of the reduced density matrix
eigenvalues = np.linalg.eigvalsh(rho_A)

# Filter out eigenvalues close to zero
#filtered_eigenvalues = eigenvalues[np.abs(eigenvalues) > 1e-12]

# Compute the logarithm of the filtered eigenvalues
#log_eigenvalues = np.log2(filtered_eigenvalues)
log_eigenvalues = np.log2(eigenvalues)

# Compute the von Neumann entropy using the filtered eigenvalues
#entropy = -np.sum(filtered_eigenvalues * log_eigenvalues)
entropy = -np.sum(eigenvalues * log_eigenvalues)

print("Von Neumann Entropy:", entropy)

Von Neumann Entropy: 0.6319259215440223


## Qiskit


In [6]:
import qiskit 

First quantum circuit, its an entangled state called the GHZ Gate. This example is from https://github.com/Qiskit/qiskit

$$
\ket{\psi_3} = \ket{000} + i\ket{111}
$$

In [7]:
from qiskit import QuantumCircuit

# We have 3 qubits, and we want to prepare its initial states to be in |000> + i|111>.
qc_GHZ = QuantumCircuit(3) # This means that we initialize 3 different qubits
qc_GHZ.h(0) # Hadamard gate, applied to qubit index (0)
qc_GHZ.p(np.pi/2,0) # p here refers to the quantum phase of the qubit. So we're adding a phase of pi/2 to the state. The second position
                    # refers to which qubit you want this phase to be applied to, which here we're applying a phase of pi/2 to the '0' qubit.
qc_GHZ.cx(0, 1) # cx refers to a CNOT gate. The term means that the '0' qubit controls the NOT gate of the '1' qubit
qc_GHZ.cx(0, 2) # cx refers to a CNOT gate. The term now means that the '0' qubit controls the NOT gate of the '2' qubit 

# We can also write the last two lines of code as
# qc_GHZ.cx(0, range(1, 2)) or more than 2 if we use more than 3 qubits. Such as if we use 5 qubits, then we would need to apply a CNOT gate from '0' qubit
# to the rest of the qubit written as qc_GHZ.cx(0, range(1, 5))

In [8]:
X = range(1,5)
for i in X:
    print(i)

1
2
3
4


we can draw the circuit using the function `.draw()`

In [9]:
qc_GHZ.draw()

┌───┐┌────────┐          
q_0: ┤ H ├┤ P(π/2) ├──■────■──
     └───┘└────────┘┌─┴─┐  │  
q_1: ───────────────┤ X ├──┼──
                    └───┘┌─┴─┐
q_2: ────────────────────┤ X ├
                         └───┘

Next that we have the quantum circuit initialized, we can start to choose which primitive function we want to use. Here we use `sampler` with the function of `measure_all(inplace = False)` to get a copy of all the circuits which all the qubits are measured

In [10]:
# We now add a classical output in a form of a measurement of all the qubits in the circuits
qc_measured = qc_GHZ.measure_all(inplace=False) # Inplace means that it will replace the circuits with a new one after measurements


We have both the initial quantum circuits with its initial states, and have a classical output in a form of a measurement of all the qubits. We can now execute the sampler primitive, and run multiple measurements using the sampler function.

In [11]:
from qiskit.primitives.sampler import Sampler
sampler = Sampler()
job = sampler.run(qc_measured, shots = 1000)
results = job.result()
print(f" > Quasi probability distribution: {results.quasi_dists}")

 > Quasi probability distribution: [{0: 0.508, 7: 0.492}]


This run gives an output of |000> close to 50% of the time and |111> close to 50% of the time up to statistical fluctuations. Next we explore the use of an estimator by first using a quantum information toolbox of $XXY + XYX + YXX - YYY$ and pass it through a `run()` function. We will use our previous circuit of `gc_GHZ`.

The first step is to define the observables we want to measure. Which in this case we want to measure the expectation value of measuring a 3 qubit system. 

In [12]:
from qiskit.quantum_info import SparsePauliOp
operator = SparsePauliOp.from_list([("XXY", 1), ("XYX", 1), ("YXX", 1), ("YYY", -1)])

# 3. Execute using the Estimator primitive
from qiskit.primitives import Estimator
estimator = Estimator()
job = estimator.run(qc_GHZ, operator, shots=1000)
result = job.result()
print(f" > Expectation values: {result.values}")

 > Expectation values: [4.]


Experimenting with different value of 1/-1 for the operator will give out a different output/results.

## A simple N-qubit GHZ gate

In [13]:
from qiskit.quantum_info import Statevector

In [14]:
def N_qubit_GHZ(n, no_simulation):
    qc_nGHZ = QuantumCircuit(n) 
    qc_nGHZ.h(0) 
    for i in range(n-1):
        qc_nGHZ.cx(i, i+1)
    print(qc_nGHZ.draw())

    qc_nGHZ_state = Statevector(qc_nGHZ)
    print(qc_nGHZ_state)

    qc_measured = qc_nGHZ.measure_all(inplace=False)
    sampler = Sampler()
    job = sampler.run(qc_measured, shots = no_simulation)
    results = job.result()
    print(f" > Quasi probability distribution: {results.quasi_dists}")

    #final_index = (2**n)-1
    #print(final_index)

    #Prob_state_00 = (results.quasi_dists[0])[0]
    #Prob_state_11 = (results.quasi_dists[0])[final_index]

    #Entropy = entropy([[Prob_state_00, 0], [0, Prob_state_11]])
    #print(f" > Entanglement entropy of the first Bell state: {Entropy}")
N_qubit_GHZ(4, 1000)


     ┌───┐               
q_0: ┤ H ├──■────────────
     └───┘┌─┴─┐          
q_1: ─────┤ X ├──■───────
          └───┘┌─┴─┐     
q_2: ──────────┤ X ├──■──
               └───┘┌─┴─┐
q_3: ───────────────┤ X ├
                    └───┘
Statevector([0.70710678+0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.70710678+0.j],
            dims=(2, 2, 2, 2))
 > Quasi probability distribution: [{0: 0.518, 15: 0.482}]


A better code for n-qubit GHZ gate are from https://docs.quantum.ibm.com/guides/hello-world

In [15]:
from qiskit import QuantumCircuit
 
def get_qc_for_n_qubit_GHZ_state(n: int) -> QuantumCircuit:
    """This function will create a qiskit.QuantumCircuit (qc) for an n-qubit GHZ state.
 
    Args:
        n (int): Number of qubits in the n-qubit GHZ state
 
    Returns:
        QuantumCircuit: Quantum circuit that generate the n-qubit GHZ state, assuming all qubits start in the 0 state
    """
    if isinstance(n, int) and n >= 2:
        qc = QuantumCircuit(n)
        qc.h(0)
        for i in range(n-1):
            qc.cx(i, i+1)
    else:
        raise Exception("n is not a valid input")
    return qc
 
# Create a new circuit with two qubits (first argument) and two classical
# bits (second argument)
n = 4
qc = get_qc_for_n_qubit_GHZ_state(n)

## Sparse Pauli Operator

https://docs.quantum.ibm.com/guides/operators-overview


In [16]:
op_test = SparsePauliOp.from_sparse_list(
    [('ZZ', [0, 1], 1 + 2j)], num_qubits=3
)
print(op_test)

op1 = SparsePauliOp.from_sparse_list(
    [("ZX", [1, 4], 1.0), ("YY", [0, 3], -1 + 1j)], num_qubits=5
)
print(op1)


SparsePauliOp(['IZZ'],
              coeffs=[1.+2.j])
SparsePauliOp(['XIIZI', 'IYIIY'],
              coeffs=[ 1.+0.j, -1.+1.j])


# Observables with Pauli

https://docs.quantum.ibm.com/guides/specify-observables-pauli

Observables $\rightarrow$ set of physical properties that can be measured. Such as spin, position, momentum, energy.
Measuring $n$-qubit observable of $\mathcal{O}$ on a quantum computers need to be represented by sum of products of Pauli operators.

$$
\mathcal{O} = \sum_{k=1}^{K} \alpha_k P_k, P_k \in \{I, X, Y, Z\}^{\otimes n}, \alpha_k \in \R.
$$

where $I$, $X, Y, Z$ as identity matrices, and x, y, z Pauli matrices respectively.
We can try to write down the Hamiltonian for a 1+1 dimensional Ising model of a $-\frac{1}{2}$ system as a combination of Paulis bases.
The Hamiltonian for the Ising model,
$$
H = \sum_{\braket{i,j}} Z_i Z_j - \sum_{i=1}^{n} X_i.
$$

We will first study a 1-dimensional Ising chain.

In [17]:
from qiskit.quantum_info import SparsePauliOp
# Define how many qubits you want to check out
n=5

# Define the interaction of the Ising model
ising_interaction = [("ZZ", [i, i+1], 1) for i in range(n-1)]
ising_external_field = [("X", [i], -1) for i in range(n)]

print(ising_interaction)
print(ising_external_field)

ising_hamiltonian = SparsePauliOp.from_sparse_list(ising_interaction + ising_external_field, num_qubits=n)
ising_hamiltonian


[('ZZ', [0, 1], 1), ('ZZ', [1, 2], 1), ('ZZ', [2, 3], 1), ('ZZ', [3, 4], 1)]
[('X', [0], -1), ('X', [1], -1), ('X', [2], -1), ('X', [3], -1), ('X', [4], -1)]


SparsePauliOp(['IIIZZ', 'IIZZI', 'IZZII', 'ZZIII', 'IIIIX', 'IIIXI', 'IIXII', 'IXIII', 'XIIII'],
              coeffs=[ 1.+0.j,  1.+0.j,  1.+0.j,  1.+0.j, -1.+0.j, -1.+0.j, -1.+0.j, -1.+0.j,
 -1.+0.j])

Using qiskit to write a matrix in terms of Pauli matrices

In [18]:
import numpy as np
from qiskit.quantum_info import SparsePauliOp
 
matrix = np.array([[-1, 0, 0.5, -1],
		   [0, 1, 1, 0.5],
		   [0.5, 1, -1, 0],
		   [-1, 0.5, 0, 2]])
 
observable = SparsePauliOp.from_operator(matrix)
print(observable)  

SparsePauliOp(['II', 'IZ', 'XI', 'YY', 'ZI', 'ZZ'],
              coeffs=[ 0.25+0.j, -1.25+0.j,  0.5 +0.j,  1.  -0.j, -0.25+0.j,  0.25+0.j])


where then teh operator are written as follows,

$$
\mathcal{O} = 0.25 - 1.25Z_1 + 0.5X_2 + Y_2Y_1 - 0.25Z_2 
$$

Another example from https://github.com/Qiskit/textbook/blob/main/notebooks/ch-gates/multiple-qubits-entangled-states.ipynb

In [19]:
from qiskit import QuantumCircuit, assemble
import numpy as np
from qiskit.visualization import plot_histogram, plot_bloch_multivector

This circuits are from https://github.com/Qiskit/qiskit-tutorials/blob/master/tutorials/circuits_advanced/03_advanced_circuit_visualization.ipynb

In [20]:
# Quantum circuits with 3 quantum registers and 3 classical 
quantum_circuit = QuantumCircuit(3, 3)

quantum_circuit.x(1)
quantum_circuit.h(range(3))
quantum_circuit.cx(0, 1)
quantum_circuit.measure(range(3), range(3))
quantum_circuit.draw()

┌───┐          ┌─┐   
q_0: ┤ H ├───────■──┤M├───
     ├───┤┌───┐┌─┴─┐└╥┘┌─┐
q_1: ┤ X ├┤ H ├┤ X ├─╫─┤M├
     ├───┤└┬─┬┘└───┘ ║ └╥┘
q_2: ┤ H ├─┤M├───────╫──╫─
     └───┘ └╥┘       ║  ║ 
c: 3/═══════╩════════╩══╩═
            2        0  1

# Bell states

This code is from https://quantumcomputinguk.org/tutorials/introduction-to-bell-states.
We need to make these 4 states in qiskit.

$$
\ket{\Psi_{00}^+} = \frac{\ket{00} + \ket{11}}{\sqrt{2}}
$$
$$
\ket{\Psi_{01}^-} = \frac{\ket{00} - \ket{11}}{\sqrt{2}}
$$
$$
\ket{\Psi_{10}^+} = \frac{\ket{01} + \ket{10}}{\sqrt{2}}
$$
$$
\ket{\Psi_{11}^-} = \frac{\ket{01} - \ket{10}}{\sqrt{2}}
$$

The first bell state can be created using a Hadamard gate and a CNOT gate

In [21]:
from qiskit.quantum_info import entropy

In [22]:
from qiskit.quantum_info import Statevector
def Bell_State_1():
    Bell_1 = QuantumCircuit(2)
    Bell_1.h(0)
    Bell_1.cx(0, 1)
    print(Bell_1.draw())

    Bell_1_State = Statevector(Bell_1)
    print(Bell_1_State)

    Bell_1_measure = Bell_1.measure_all(inplace=False)

    from qiskit.primitives.sampler import Sampler
    sampler = Sampler()
    job = sampler.run(Bell_1_measure, shots = 1000)
    results = job.result()
    print(f" > Quasi probability distribution of the first Bell state: {results.quasi_dists}")

    Prob_state_00 = (results.quasi_dists[0])[0]
    Prob_state_11 = (results.quasi_dists[0])[3]

    Entropy = entropy([[Prob_state_00, 0], [0, Prob_state_11]])
    print(f" > Entanglement entropy of the first Bell state: {Entropy}")

Bell_State_1()

     ┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘
Statevector([0.70710678+0.j, 0.        +0.j, 0.        +0.j,
             0.70710678+0.j],
            dims=(2, 2))
 > Quasi probability distribution of the first Bell state: [{0: 0.525, 3: 0.475}]
 > Entanglement entropy of the first Bell state: 0.99819587904281


In [23]:
def Bell_State_2():
    Bell_2 = QuantumCircuit(2)
    Bell_2.x(0)
    Bell_2.h(0)
    Bell_2.cx(0, 1)
    print(Bell_2.draw())

    Bell_2_State = Statevector(Bell_2)
    print(Bell_2_State)

    Bell_2_measure = Bell_2.measure_all(inplace=False)

    from qiskit.primitives.sampler import Sampler
    sampler = Sampler()
    job = sampler.run(Bell_2_measure, shots = 1000)
    results = job.result()
    print(f" > Quasi probability distribution of the second Bell state: {results.quasi_dists}")
    Prob_state_00 = (results.quasi_dists[0])[0]
    Prob_state_11 = (results.quasi_dists[0])[3]

    Entropy = entropy([[Prob_state_00, 0], [0, Prob_state_11]])
    
    print(f" > Entanglement entropy of the second Bell state: {Entropy}")
    
Bell_State_2()

     ┌───┐┌───┐     
q_0: ┤ X ├┤ H ├──■──
     └───┘└───┘┌─┴─┐
q_1: ──────────┤ X ├
               └───┘
Statevector([ 0.70710678+0.j,  0.        +0.j,  0.        +0.j,
             -0.70710678+0.j],
            dims=(2, 2))
 > Quasi probability distribution of the second Bell state: [{0: 0.552, 3: 0.448}]
 > Entanglement entropy of the second Bell state: 0.992183779438731


In [24]:
def Bell_State_3():
    Bell_3 = QuantumCircuit(2)
    Bell_3.h(0)
    Bell_3.x(1)
    Bell_3.cx(0,1)
    print(Bell_3.draw())

    Bell_3_State = Statevector(Bell_3)
    print(Bell_3_State)

    Bell_3_measure = Bell_3.measure_all(inplace=False)

    from qiskit.primitives.sampler import Sampler
    sampler = Sampler()
    job = sampler.run(Bell_3_measure, shots = 1000)
    results = job.result()
    print(f" > Quasi probability distribution of the third Bell state: {results.quasi_dists}")
    Prob_state_00 = (results.quasi_dists[0])[1]
    Prob_state_11 = (results.quasi_dists[0])[2]

    Entropy = entropy([[Prob_state_00, 0], [0, Prob_state_11]])
    
    print(f" > Entanglement entropy of the third Bell state: {Entropy}")

Bell_State_3()

     ┌───┐     
q_0: ┤ H ├──■──
     ├───┤┌─┴─┐
q_1: ┤ X ├┤ X ├
     └───┘└───┘
Statevector([0.        +0.j, 0.70710678+0.j, 0.70710678+0.j,
             0.        +0.j],
            dims=(2, 2))
 > Quasi probability distribution of the third Bell state: [{1: 0.515, 2: 0.485}]
 > Entanglement entropy of the third Bell state: 0.9993506898146103


Turns out they have a `Statevector` function that translate the quantum circuit into a statevector.

In [25]:
from matplotlib.figure import Figure
import matplotlib_inline
from qiskit.quantum_info import Statevector

def Bell_State_4():
    Bell_4 = QuantumCircuit(2)
    Bell_4.h(0)
    Bell_4.x(1)
    Bell_4.z(0)
    Bell_4.z(1)
    Bell_4.cx(0, 1)
    print(Bell_4.draw())

#    print(plot_bloch_multivector(Bell_4_State, title='4th Bell State', reverse_bits=True))

    Bell_4_measure = Bell_4.measure_all(inplace=False)

    Bell_4_State = Statevector(Bell_4)
    print(Bell_4_State)
    from qiskit.primitives.sampler import Sampler
    sampler = Sampler()
    job = sampler.run(Bell_4_measure, shots = 1000)
    results = job.result()
    print(f" > Quasi probability distribution of the third Bell state: {results.quasi_dists}")

    Prob_state_00 = (results.quasi_dists[0])[1]
    Prob_state_11 = (results.quasi_dists[0])[2]

    Entropy = entropy([[Prob_state_00, 0], [0, Prob_state_11]])
    Bell_4_Visualize = plot_bloch_multivector([[Prob_state_00, 0], [0, Prob_state_11]])

    #Entropy = entropy(Bell_4_State), the statevector function gives back the state of the system following the quantum circuit.
    print(f" > Entanglement entropy of the fourth Bell state: {Entropy}")

Bell_State_4()

     ┌───┐┌───┐     
q_0: ┤ H ├┤ Z ├──■──
     ├───┤├───┤┌─┴─┐
q_1: ┤ X ├┤ Z ├┤ X ├
     └───┘└───┘└───┘
Statevector([ 0.        +0.j,  0.70710678+0.j, -0.70710678+0.j,
              0.        +0.j],
            dims=(2, 2))
 > Quasi probability distribution of the third Bell state: [{1: 0.492, 2: 0.508}]
 > Entanglement entropy of the fourth Bell state: 0.9998153271549207


The previous work can also be done using density matrix, and you can convert the quantum circuit using `DensityMatrix` function. I am still working out on why the figure wont show up.

There are many tools qiskit offers for visualization.

In [26]:
qc_TEST = QuantumCircuit(2)
qc_TEST.h(0)
qc_TEST.x(1)
qc_TEST.cx(0, 1)
qc_TEST.x(1)
qc_TEST.draw()

┌───┐          
q_0: ┤ H ├──■───────
     ├───┤┌─┴─┐┌───┐
q_1: ┤ X ├┤ X ├┤ X ├
     └───┘└───┘└───┘

# SYK MODEL

The SYK model is a 0+1 dimensional model of $N \gg 1$ fermions with an all-to-all random quartic interaction. The mild generalization of a q-body Hamiltonian for this model is written as follows,

$$
H = i^{q/2} \sum_{1 \le i_1 < \ldots < i_q \le N} J_{i_1 \ldots i_q} \psi_{i_1}\ldots\psi_{i_q},
$$
with a variance of zero, this means that the Gaussian couplings is now,
$$
\sigma = \sqrt{(q-1)!} \frac{J}{N^{\frac{q-1}{2}}}
$$
with $ q $ as an integer. Lets try an SYK model with $q=4$, with the coupling constant following a normal Gaussian distribution with mean of $\mu = 0$ and the variance as $\sigma =\sqrt{(q-1)!}J/N^{\frac{q-1}{2}}$



## Recursion relation for the representation matrices

This recursion relations are mentioned here, https://arxiv.org/pdf/1711.08482.

\begin{equation}
\begin{split}
\psi_i^{(K)} &= \psi_i^{(K-1)} \otimes \begin{pmatrix} -1 & 0 \\ 0 & 1 \end{pmatrix}, i =1,\cdots, N-2 \\

\psi_{N-1}^{(K)} &= \frac{1}{\sqrt{2}} I_{2^{K-1}} \otimes \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix}, \\

\psi_N^{(K)} &= \frac{1}{\sqrt{2}} I_{2^{K-1}} \otimes \begin{pmatrix} 0 & -i \\ i & 0 \end{pmatrix},
\end{split}
\end{equation}

with $K=N/2$. The following recursion relation can be computed using numpy and matplotlib to visualize the result.
Lets say that we have 4 Majorana fermions, that means $q=4$, and that we choose $N=20$, as 



## Simple State

Lets start our disussion from a simple 2 qubit state

$$
\ket{\psi} = \frac{1}{\sqrt{2}} (\ket{00} + \ket{11}),
$$

where, $\ket{00} = \ket{0}\otimes\ket{0} = \begin{pmatrix} 1 \\ 0 \end{pmatrix} \otimes \begin{pmatrix} 1 \\ 0 \end{pmatrix} = \begin{pmatrix} 1 \\ 0 \\ 0 \\ 0 \end{pmatrix}$, where each of the component is a qubit, which can have 2 values, either a 0 or a 1.

and so in python we would write this state as,

In [27]:
state_00_ket = np.array([[1], [0], [0], [0]], dtype=np.float64)
state_00_ket *= 1/np.sqrt(2)
print(f"state 00 :\n", state_00_ket) 
state_11_ket = np.array([[0], [0], [0], [1]], dtype=np.float64)
state_11_ket *= 1/np.sqrt(2)
print(f"state 00 :\n", state_11_ket) 

psi_ket = state_00_ket + state_11_ket
print(psi_ket)

state 00 :
 [[0.70710678]
 [0.        ]
 [0.        ]
 [0.        ]]
state 00 :
 [[0.        ]
 [0.        ]
 [0.        ]
 [0.70710678]]
[[0.70710678]
 [0.        ]
 [0.        ]
 [0.70710678]]


where the dtype = float64 can store up to around 17 decimal points, where typically consists of a sign bit, 11 bits exponent, 52 bits mantissa. Then it is straightforward to calculate the density matrices from this state,
$$
\begin{equation}
\begin{split}
\rho &= \sum_i \ket{\psi_i}\bra{\psi_i} = \bigg(\frac{1}{\sqrt{2}}\ket{00}\bra{00}\bigg) + \bigg(\frac{1}{\sqrt{2}}\ket{11}\bra{11}\bigg)\\
&= \frac{1}{2} (\ket{00}\bra{00} + \ket{11}\bra{11})
\end{split}
\end{equation}
$$

we will make use of a function from numpy called `np.matrix.getH()`, which stands for 'get Hermitian'. For now, lets calculate it straight from the equation and we'll write the generelized function later on. For a normal matrix multiplication, we can use numpy's `np.matmul(x1, x2)` with `x1 x2` as the matrices.

In [28]:
state_00_bra = np.matrix.getH(state_00_ket)
state_11_bra = np.matrix.getH(state_11_ket)

density_matrix = np.matmul(state_00_ket, state_00_bra) + np.matmul(state_11_ket, state_11_bra)
density_matrix

array([[0.5, 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.5]])

Our ultimate goal here is to calculate the entanglement entropy given by the following equaion,
$$
\rho_A = -Tr_B[\rho \log{\rho}],
$$
and it involves doing 2 types of operation on the matrix, first is taking a log of a matrix and the second is taking a partial trace out with respect of subsystem B. We will first figure out how to perform a log on a matrix in python with the help of numpy or scipy.

Now, in Scipy, they have a function called `scipy.linalg.logm(A)`, which is the matrix logarithm and is the inverse of `expm`, where `expm(logm(A)) == A`.

In [29]:
import scipy as scp
from scipy.linalg import logm

In [30]:
log_density_matrix = logm(density_matrix)
log_density_matrix

c:\Users\Acer Aspire 5\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\linalg\_matfuncs_inv_ssq.py:827: LogmExactlySingularWarning: The logm input matrix is exactly singular.
  warnings.warn(exact_singularity_msg, LogmExactlySingularWarning)


array([[ -0.69314718,   0.        ,   0.        ,   0.        ],
       [  0.        , -46.05170186,   0.        ,   0.        ],
       [  0.        ,   0.        , -46.05170186,   0.        ],
       [  0.        ,   0.        ,   0.        ,  -0.69314718]])

We then mutliply it with $\rho$ again to get $\rho \log{\rho}$,


In [31]:
rho_log_rho = np.matmul(density_matrix, log_density_matrix)
rho_log_rho

array([[-0.34657359,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , -0.34657359]])

And so for our simple 2 qubit model, we can then calculate the entanglement entropy as,

In [32]:
entanglement_entropy_2_qubit = -np.trace(rho_log_rho)
entanglement_entropy_2_qubit

0.6931471805599454

Now that we have a general idea of what we should do to calculate the entanglement entropy, we can start by incorporating the SYK model to the operation.

$$
H = i^{q/2} \sum_{1 \le i_1 < \ldots < i_q \le N} J_{i_1 \ldots i_q} \psi_{i_1}\ldots\psi_{i_q},
$$
with a variance of zero, this means that the Gaussian couplings is now,
$$
\sigma = \sqrt{(q-1)!} \frac{J}{N^{\frac{q-1}{2}}}
$$

where we will consider 4 interacting fermions, with the coupling constant taken from a Gaussian ensemble with a mean of $\mu = 0$ and hence a variance of $\sigma = \sqrt{3!} \frac{J}{N^{3/2}}$.

$$
H = \sum_{1\le i < j < j < k \le N}^{N} \psi_i \psi_j \psi_k \psi_l.
$$
The matrix size of the coupling should be $2^{N/2} \times 2^{N/2}$.

I will first follow the flow of Prof. Junggi and Vinay's work on mathematica, and then find if I can do things differently (if possible) that could improve the codes.

First, we define a function that generates gamma matrices, they call `gengammafast(n)`.

https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html

## `GenGammaFast` Function

In [33]:
pauli_x = csr_matrix(np.array([[0, 1], [1, 0]])).toarray()
print(pauli_x)
repeat_pauli_x = np.repeat(pauli_x, 8)
print(repeat_pauli_x)
id_matrix = scp.sparse.identity(8).toarray()
print(id_matrix)

NameError: name 'csr_matrix' is not defined

In [ ]:
#TESTING
import numpy as np
from scipy.sparse import csr_matrix, kron
from scipy.sparse import identity

def gengammafast(n):
    # Define Pauli matrices and identity matrix
    pauli_x = csr_matrix(np.array([[0, 1], [1, 0]])).toarray()
    pauli_y = csr_matrix(np.array([[0, -1j], [1j, 0]])).toarray()
    pauli_z = csr_matrix(np.array([[1, 0], [0, -1]])).toarray()
    #identity_matrix = csr_matrix(np.array([[1, 0], [0, 1]]))
    identity_matrix = identity(n).toarray()

    result = []

    dum1 = pauli_z * (n-1)
    dum1 = kron(dum1, pauli_x)

    dum2 = pauli_z * (n-1)
    dum2 = kron(dum2, pauli_y)

    for i in range(n):
        # Use Kronecker product to construct the required matrix
        kron_prod1 = dum1[0]
        for mat in dum1[1:]:
            kron_prod1 = kron(kron_prod1, mat)
        result.append(kron_prod1)
        
        kron_prod2 = dum2[0]
        for mat in dum2[1:]:
            kron_prod2 = kron(kron_prod2, mat)
        result.append(kron_prod2)
        
        # Update dum1 and dum2
        dum1 = dum1[1:] + [identity_matrix]
        dum2 = dum2[1:] + [identity_matrix]

    return result

# Example usage
n = 4
result = gengammafast(n)
print(len(result))
for matrix in result:
    print(matrix.toarray())  # Print dense form for readability


TypeError: expected dimension <= 2 array or matrix

In [ ]:
import numpy as np
from scipy.sparse import csr_matrix, kron
from scipy.sparse import identity

def gengammafast(n):
    # Define Pauli matrices and identity matrix
    sig1 = csr_matrix(np.array([[0, 1], [1, 0]]))
    sig2 = csr_matrix(np.array([[0, -1j], [1j, 0]]))
    sig3 = csr_matrix(np.array([[1, 0], [0, -1]]))
    id = csr_matrix(np.array([[1, 0], [0, 1]]))
    
    result = []

    dum1 = [sig3] * (n - 1) + [sig1]
    dum2 = [sig3] * (n - 1) + [sig2]

    for i in range(n):
        # Use Kronecker product to construct the required matrix
        kron_prod1 = dum1[0]
        for mat in dum1[1:]:
            kron_prod1 = kron(kron_prod1, mat)
        result.append(kron_prod1)
        
        kron_prod2 = dum2[0]
        for mat in dum2[1:]:
            kron_prod2 = kron(kron_prod2, mat)
        result.append(kron_prod2)
        
        # Update dum1 and dum2
        dum1 = dum1[1:] + [id]
        dum2 = dum2[1:] + [id]

    return result

# Example usage
n = 2
result = gengammafast(n)
for matrix in result:
    print(matrix.toarray())  # Print dense form for readability


[[ 0  1  0  0]
 [ 1  0  0  0]
 [ 0  0  0 -1]
 [ 0  0 -1  0]]
[[0.+0.j 0.-1.j 0.+0.j 0.+0.j]
 [0.+1.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+1.j]
 [0.+0.j 0.+0.j 0.-1.j 0.+0.j]]
[[0 0 1 0]
 [0 0 0 1]
 [1 0 0 0]
 [0 1 0 0]]
[[0.+0.j 0.+0.j 0.-1.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.-1.j]
 [0.+1.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+1.j 0.+0.j 0.+0.j]]


## `GenRandomCoupling4` Function

Then, we define a function that generates random coupling value from a normal (Gaussian) distribution with a mean of 0 and variance of 1

In [ ]:
def genrandomcoupling4(n, var=1, mean=0):
    # Generate a 4-dimensional array for each of the qubits we used with random values from a normal distribution.
    # Note that we're considering a q=4 SYK model.
    result = np.random.normal(loc=mean, scale=var, size=(2*n, 2*n, 2*n, 2*n))
    return result

# Example usage
n = 2
var = 1
mean = 0
coupling_array_example = genrandomcoupling4(n, var, mean)
print(coupling_array_example[0])
print(len(coupling_array_example))

[[[-0.95535991 -0.74764813 -0.27379232  0.64342986]
  [-0.01441041  0.04914075 -1.25122972 -0.83858134]
  [ 0.32584145  0.54105748 -1.67725792 -1.08890423]
  [-0.17557002  1.23415509 -1.81305106  0.33238948]]

 [[-0.9556085  -0.28430025  0.97733597  0.12873729]
  [ 0.2777473   0.55238811  0.52831097 -1.86305596]
  [ 0.19448333 -0.88134047  1.82829407 -0.6747165 ]
  [ 0.15105276  1.07892265 -1.49906832  1.82264524]]

 [[-0.14706167 -0.66246316 -0.1006622  -1.43070186]
  [-0.53375975 -0.42529815  1.88200071 -0.08039174]
  [-1.14547736  0.37448111 -1.14797349 -0.67715446]
  [-1.02441159  0.81794306  0.9624273  -0.77719989]]

 [[ 0.02056569 -0.24529231 -0.95291606  0.94138687]
  [-0.2278445  -0.45792749 -1.77812827 -1.83776724]
  [ 1.70429396  0.85567415 -1.81488774 -1.20220708]
  [-1.50274403 -0.35630726  0.78446296  0.03159643]]]
4


## `GenHam4` function

In [ ]:
# Then we write the Hamiltonians for the SYK model

def genham4(n, coupling, gamma):
    hamiltonian_initial = 0  # Initialize the Hamiltonian to zero

    # Calculate the Hamiltonian using nested loops
    for i1 in range(2 * n):
        for i2 in range(i1 + 1, 2 * n):
            for i3 in range(i2 + 1, 2 * n):
                for i4 in range(i3 + 1, 2 * n):
                    term = (np.sqrt(6) ** 1 * n ** (-3 / 2)) * coupling[i1][i2][i3][i4]
                    term *= np.dot(gamma[i1], np.dot(gamma[i2], np.dot(gamma[i3], gamma[i4])))
                    hamiltonian_initial += term

    return hamiltonian_initial

# Example usage
n = 4

# Define a sample coupling tensor from the function of genrandom
coupling_example = genrandomcoupling4(n, 1, 0)
print(coupling_example)

# Define sample gamma matrices from the function of gengammafast
gamma_example = gengammafast(n)
for matrix in gamma_example:
    print(matrix.toarray())

# Calculate the Hamiltonian
hamiltonian_example = genham4(n, coupling_example, gamma_example)
print(hamiltonian_example.toarray())


[[[[-1.02386768e+00  2.77545906e-01  1.21231459e+00 ...  6.43559659e-01
    -2.11720715e+00 -1.65983460e-02]
   [ 5.40767374e-01 -2.55573356e-01 -3.57134094e-01 ...  9.07797664e-02
    -1.42443221e+00  2.57303501e-01]
   [ 1.58101587e+00  1.67382883e+00  1.08000174e+00 ... -1.17343864e+00
     1.43547876e+00 -1.68748511e+00]
   ...
   [ 5.54359591e-01 -2.94233146e-02 -4.78153504e-01 ... -8.92459890e-01
     2.72491353e-01  1.31619907e+00]
   [-8.38433761e-02  2.50664410e-01  2.14709368e+00 ...  2.83657423e-01
     1.27770208e+00  3.52630603e-01]
   [ 2.44860078e+00 -2.22982560e+00  3.09129064e-01 ...  2.80707314e-01
     2.44557735e+00  3.07493118e+00]]

  [[-9.83430110e-02  1.14033388e+00  8.32483654e-01 ...  1.23617011e+00
    -2.36024286e-01  3.87288754e-01]
   [-2.26594230e-01 -4.73036745e-02  2.16602225e+00 ... -6.72888952e-01
    -3.05915978e-01 -1.04730871e-01]
   [ 7.77369067e-01  3.91496458e-01 -5.24649594e-01 ...  6.57384945e-01
     1.08053872e+00  1.48716017e+00]
   ...
   

Following Prof. Junggi and Vinay's code, they implemented a swap function to swap 2 positions with each other.
This one can be implemented simply enough by switching 

Then they implemented a dTraceSystem to do a partial trace of a matrix. 
The function `d_trace_system` essentially performs the following operations:

1. Sorts the subsystems (qudits) to be traced out.
2. Iterates over each qudit, performing a series of matrix permutations and partial traces.
3. Uses nested loops to manage the swaps and partial traces.
4. Returns the final reduced density matrix after tracing out the specified qudits.

In [ ]:
import numpy as np

def swap_parts(expr, pos1, pos2):
    expr[pos1], expr[pos2] = expr[pos2], expr[pos1]
    return expr

# Example usage:
expr = np.array([1, 2, 3, 4])
print(swap_parts(expr, 1, 2))  # Swap elements at positions 1 and 2


[1 3 2 4]


Do note that in Python, the index starts from [0], and so position 1 and 2 really means 2nd and 3rd position of the matrix.

In [ ]:
from scipy.sparse import csr_matrix

def d_trace_system(D, s, dimension_):
    Qudits = sorted(s, reverse=True)
    TrkM = D
    dim = dimension_
    z = len(Qudits) + 1

    for q in range(z - 1):
        n = int(np.log(TrkM.shape[0]) / np.log(dim))
        M = TrkM
        k = Qudits[q]

        if k == n:
            TrkM = []
            for p in range(0, dim**n, dim):
                TrkM.append(sum(M[p+h, :] for h in range(dim)))
            TrkM = np.array(TrkM)
        else:
            for j in range(n - k):
                b = [0]
                for i in range(1, dim**n + 1):
                    if (i - 1) // dim**(n-1) % dim != (i - 1) // dim**(n-j-1) % dim and i not in b:
                        swapped_index = (i - 1) // dim**(n-1) * dim**(n-1) + (i - 1) % dim**(n-1)
                        b.append(swapped_index + 1)
                        c = np.arange(1, dim**n + 1)
                        perm = swap_parts(c, i, swapped_index + 1)
                        M = M[perm-1][:, perm-1]
            TrkM = []
            for p in range(0, dim**n, dim):
                TrkM.append(sum(M[p+h, :] for h in range(dim)))
            TrkM = np.array(TrkM)

    return TrkM

# Example usage:
D_example = np.random.rand(16, 16)  # Example density matrix
print(D_example[15, :])
s_example = [2, 1]
dimension_example = 2
#print(D)

TrkM = d_trace_system(D_example, s_example, dimension_example)
print(TrkM)


[0.20343403 0.23601344 0.20546133 0.03991528 0.15831796 0.24055896
 0.97662745 0.27307941 0.41208492 0.07591906 0.0179587  0.02188838
 0.75077914 0.4435761  0.36149202 0.48532117]
[[1.42439384 1.5216268  2.4145432  1.78869687 1.83395787 2.87889584
  1.27778958 0.90333787]
 [1.90801361 1.88872527 1.95778927 1.86113159 1.73669026 1.5535986
  1.93133969 2.10691304]
 [1.8454899  2.12915767 2.12770862 1.93665735 2.09395931 2.73394123
  0.93164724 1.69380142]
 [1.65433533 1.87834718 0.50674632 1.52289807 1.97273463 2.18491655
  2.52170075 1.68089311]]


Turns out python have its own matrix multiplication with the '@' symbol, which apparently is supposed to be more concise than that of numpy's matmul function, nevertheless, both function works and returns the same result. I'm not sure which one is better but for now, we will try the '@' operator from python.

In [ ]:
import numpy as np
from scipy.linalg import expm, eigh

# Assume gengammafast, genrandomcoupling4, and genham4 are defined as in previous explanations

# Step 1: Define gamma
largen = 8  # Replace with the appropriate value for largen
gamma = gengammafast(largen)

# Step 2: Generate random couplings with largen size
coupling40 = genrandomcoupling4(largen)

# Step 3: Compute the Hamiltonian using the genham40 function
ham40 = genham4(largen, coupling40, gamma)

# Step 4: Define Uq0, which i think works to constructs the time evolution operator for the quantum system described by ham40.
#         I am still not sure why the parameter i0 was added in the first place, but for now I will follow their functions.
def Uq0(ham, i0, t):
    eigvals, eigvecs = eigh(ham)
    exp_diag = np.diag(np.exp(-1j * t * eigvals))
    return eigvecs.conj().T @ exp_diag @ eigvecs

# Step 5: Define F2, which is just a implified interface for the Uq0 function.
def F2(t):
    return Uq0(ham40, 1, t)

# Step 6: Define the initial state
state00 = np.zeros(len(ham40))
state00[0] = 1
state0 = state00.reshape(-1, 1)

# Step 7: Define Psi_nd0, 
def Psi_nd0(t):
    return F2(t) @ state0

# Step 8: Define state
def state(t):
    psi_t = Psi_nd0(t)
    return psi_t @ psi_t.conj().T

# Step 9: Compute the evolution of the state
delta = 0.01
rho_AB = []
for i in range(1, 101):
    rho_AB.append(state(i * delta))

# Example output
print(rho_AB)


ValueError: dimension mismatch

# Calculating the EE from Random States

This part only focuses on calculating the Entanglement Entropy without the affect from the Hamiltonian.

In [1]:
import numpy as np
import scipy as scp
import qiskit
import qutip as qt

Now that we have initial states thats usually written as 
$$
\ket{\psi} = \alpha_{\nu_i}\sum_{i}^{N}\ket{\nu_1 \nu_2 \cdots \nu_N},
$$
we will need to write a code that generates the constants for each of the qubits $\nu_i$.

In [2]:
def Initial_State_Generator(nQubits, mean = 0, std = 1):
    """
    Generate a row vector of random zeros and ones, and then each of the components are multiplied by a constant taken from a Gaussian ensemble
    for each of the components of the statevector.

    Note that the '*' operation in python corresponds to a direct multiplication for each of the components of the statevector.
    Example :
    [[x11, x12],    *    [[a11, a12],
     [x21, x22]]          [a21, a22]]

    will return,
    [[x11*a11, x12*a12],
     [x21*a21, x22*a22]]

    Parameters:
    nQubits (int): The length of the vector which corresponds to how many qubits.

    Returns:
    numpy.ndarray: A row vector of random zeros and ones.
    """
    if not isinstance(nQubits, int) or nQubits <= 0:
        raise ValueError("The number of qubits must be a positive integer.")
    gaussian_ensemble = np.random.normal(mean, std, size=(2**nQubits, 1))
    #print(f"The gaussian ensemble for each of the components: \n", gaussian_ensemble)
    random_states = np.random.choice([1, 0], size=(2**nQubits, 1))
    #print(f"The random state generated: \n", random_states)
    matrix = gaussian_ensemble * random_states
    norm = np.linalg.norm(matrix)
    matrix = matrix/norm  # normalized matrix
    return matrix

In [3]:
test_state_generator_gaussian = Initial_State_Generator(8, 0, 1)
print(f"Length of the statevector with gaussian random constants :", len(test_state_generator_gaussian))
print(test_state_generator_gaussian)

Length of the statevector with gaussian random constants : 256
[[-0.05285762]
 [-0.00057306]
 [-0.1790543 ]
 [ 0.        ]
 [-0.07066254]
 [ 0.00274399]
 [ 0.        ]
 [-0.        ]
 [-0.08882708]
 [ 0.        ]
 [ 0.12181905]
 [ 0.0918395 ]
 [-0.        ]
 [-0.        ]
 [ 0.        ]
 [-0.16466738]
 [ 0.        ]
 [-0.        ]
 [-0.07072627]
 [ 0.0051886 ]
 [ 0.02659491]
 [-0.00207462]
 [-0.06903102]
 [-0.06821056]
 [ 0.02937557]
 [-0.10109383]
 [ 0.        ]
 [ 0.15059815]
 [ 0.        ]
 [-0.08028851]
 [ 0.        ]
 [ 0.21247657]
 [-0.        ]
 [ 0.        ]
 [-0.00320721]
 [-0.07432543]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [-0.05852643]
 [-0.        ]
 [ 0.08533674]
 [ 0.        ]
 [ 0.03311587]
 [ 0.        ]
 [ 0.        ]
 [-0.        ]
 [-0.        ]
 [-0.        ]
 [-0.        ]
 [-0.        ]
 [ 0.01751275]
 [ 0.        ]
 [-0.08149809]
 [-0.12233203]
 [ 0.02795533]
 [-0.        ]
 [-0.08066765]
 [ 0.01657015]
 [-0.        ]
 [-0.        ]
 [-0.  

Next step is to write a code that represents the density matrices, from the statevectors we just made,
$$
\rho = \sum_i\ket{\psi_i}\bra{\psi_i}
$$
we can write the hermitian complex conjugate using an internal function from scipy. We will make use of a function from numpy called `np.matrix.getH()`, which stands for 'get Hermitian'. For now, lets calculate it straight from the equation and we'll write the generelized function later on. For a normal matrix multiplication, we can use numpy's `np.matmul(x1, x2)` with `x1 x2` as the matrices or just use the function `@` from python that perform the same operation.

In this case, the `x1 x2` matrices are the initial states we just generated.

In [4]:
def Density_Matrix_Operator(matrix1):
    """
    Generate a density matrix, by multiplying the statevector with its Hermitian conjugate.

    Parameters:
    matrix1 (np.array): The statevector of the quantum system.

    Returns:
    numpy.ndarray: A density matrix after multiplying (matrix multiplication) the ket(psi) with bra(psi).
    """
    matrix1_Herm_conj = np.matrix.getH(matrix1) # get the hermitian conjugate of the matrix.
    dense_matrix = matrix1 @ matrix1_Herm_conj # then multiply the conjugate and the original matrix.

    return dense_matrix

In [5]:
test_density_matrix = np.array(Density_Matrix_Operator(test_state_generator_gaussian))
print(f"Shape of the density matrix :", test_density_matrix.shape)
print(len(test_density_matrix))
print(test_density_matrix)

Shape of the density matrix : (256, 256)
256
[[ 2.79392830e-03  3.02908290e-05  9.46438443e-03 ... -2.63950692e-03
   0.00000000e+00 -5.97188848e-03]
 [ 3.02908290e-05  3.28402959e-07  1.02609666e-04 ... -2.86166444e-05
   0.00000000e+00 -6.47452023e-05]
 [ 9.46438443e-03  1.02609666e-04  3.20604407e-02 ... -8.94128463e-03
   0.00000000e+00 -2.02296703e-02]
 ...
 [-2.63950692e-03 -2.86166444e-05 -8.94128463e-03 ...  2.49362046e-03
   0.00000000e+00  5.64182014e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-5.97188848e-03 -6.47452023e-05 -2.02296703e-02 ...  5.64182014e-03
   0.00000000e+00  1.27646268e-02]]


NOTE : The cell above here, takes too long of a time to compute.

Next is to apply a log into the matrix and multiply it with the density matrix, and take the trace with respect to subsystem B from the whole system.
$$
S_A = -Tr[\rho_A \log{\rho_A}],
$$
where we note that $\rho_A$ is defined as follows,

$$
\rho_A = Tr_{B}[\rho_{AB}]
$$

So, lets explore some of the methods of taking this matrix through the log operation.
One way is through the in-built function from scipy called `logm(A, disp = True)` (https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.logm.html)

And before taking the log of the matrix, we take the partial trace of the matrix to find the reduced density matrix from $\rho_A$ from $\rho_{AB}$. To do this, we will use the function from qutip, using the function of `Qobj` and `ptrace`.

In [6]:
from scipy.linalg import logm
import qutip as qt

In [7]:
#def reduced_density_matrix(density_matrix, subsystem_to_trace_over):
#    rdM =  qt.Qobj(density_matrix)
#    print(rdM)
#    O_ = len(density_matrix)
#    n_dim = np.sqrt(O_)
#    rdM.dims = [[2,2,2],[2,2,2]]
#    print(rdM)
#    rdM_reduced = rdM.ptrace(subsystem_to_trace_over)
#    return rdM_reduced

In [8]:
import numpy as np
import qutip as qt

def reduced_density_matrix_v2(density_matrix, subsystem_to_trace_over):
    # Convert the input density matrix to a QuTiP Qobj
    rdM = qt.Qobj(density_matrix)
    print("Original Qobj:\n", rdM)
    
    # Determine the size of the density matrix
    O_ = len(density_matrix)
    print(O_)
    num_qubits = int(np.log2(O_))
    print(num_qubits)
       
    # Set the dimensions of the Qobj based on num_qubits
    n_dims = [2] * num_qubits
    print(n_dims)
    rdM.dims = [n_dims, n_dims]
    
    print("Reshaped Qobj:\n", rdM)
    
    # Perform the partial trace over the specified subsystem
    rdM_reduced = rdM.ptrace(subsystem_to_trace_over)
    rdM_reduced.dims = [2,2]
    return rdM_reduced



In [9]:
test_reduced_density_matrix = reduced_density_matrix_v2(test_density_matrix, 1)
print(test_reduced_density_matrix)

Original Qobj:
 Quantum object: dims=[[256], [256]], shape=(256, 256), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 2.79392830e-03  3.02908290e-05  9.46438443e-03 ... -2.63950692e-03
   0.00000000e+00 -5.97188848e-03]
 [ 3.02908290e-05  3.28402959e-07  1.02609666e-04 ... -2.86166444e-05
   0.00000000e+00 -6.47452023e-05]
 [ 9.46438443e-03  1.02609666e-04  3.20604407e-02 ... -8.94128463e-03
   0.00000000e+00 -2.02296703e-02]
 ...
 [-2.63950692e-03 -2.86166444e-05 -8.94128463e-03 ...  2.49362046e-03
   0.00000000e+00  5.64182014e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-5.97188848e-03 -6.47452023e-05 -2.02296703e-02 ...  5.64182014e-03
   0.00000000e+00  1.27646268e-02]]
256
8
[2, 2, 2, 2, 2, 2, 2, 2]
Reshaped Qobj:
 Quantum object: dims=[[2, 2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 2, 2]], shape=(256, 256), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 2.79392830e-03  3.02908290e-05  9.46438443e-03 ...

NOTE : An error occured here when theres a infs values or NaNs in the matrix. Which needed to be solved by applying an `if` case built into the function later on. This is likely caused by a $\log{0}$ when applying the function to the reduced row echelon form of the density matrix.

A straightforward way to deal with this is to take the limit of $x \rightarrow 0$ for $x\log{x}$ using L'Hospital rule,

$$
\begin{equation}
 \lim_{x = 0} x\log{x} = \frac{\infty}{\infty} \rightarrow \lim_{x \to 0} x\log{x} = \lim_{x \to 0}\frac{\log{x}}{1/x} = \lim_{x \to 0}\frac{1/x}{-1/x^2} = \lim_{x \to 0} -x = 0
\end{equation}
$$

We can take the log of a matrix using `logm` function from `scipy.linalg` library. The last part is to take a normal trace, which we can implement using `np.trace` from,
$$
S_A = -Tr[\rho_A \log{\rho_A}].
$$

In [10]:
from scipy.linalg import logm
def Entanglement_Entropy(rdMatrix):
    rdMatrix = rdMatrix.full()
    log_rdMatrix = logm(rdMatrix)
    EE_before_trace = rdMatrix @ log_rdMatrix
    S_A_entanglement_entropy = -np.trace(EE_before_trace)
    return S_A_entanglement_entropy


In [11]:
Entanglement_Entropy(test_reduced_density_matrix)

(0.6931331566060416-0j)

## Random State EE Calculation combined

Now that we know how to calculate the EE by taking the

In [12]:
import matplotlib.pyplot as plt

In [13]:
def Random_State_EE_Function(nQubits, mean = 0, std = 1):
    """
    This function is used to generate a random statevectors and automatically calculates the entanglement entropy from each of the subsystem
    then is plotted to see how the entropy varies.

    Parameters:
    nQubits: The number of qubits we will consider in order to get the initial statevector of the quantum system.
    mean: The mean of the Gaussian ensemble that will be used to get the constant of the each of the component of the statevector
    std: The standard derivation of the Gaussian ensemble.

    Returns:
    Entanglement_Entropy (float64): A number (Real) that represents the value of entanglement entropy of the system.
    EE Plot: A plot of the value of the entanglement entropy.
    """
    Gaussian_Generated_Statevector = Initial_State_Generator(nQubits, mean, std)
    Gaussian_Density_Matrix = Density_Matrix_Operator(Gaussian_Generated_Statevector)
    Reduced_GDM_EE_List = []
    #for i in range(nQubits-1):
    for i in range(nQubits):
        Reduced_GDM = reduced_density_matrix_v2(Gaussian_Density_Matrix, i)
        Reduced_GDM_EE = Entanglement_Entropy(Reduced_GDM)
        Reduced_GDM_EE_List.append(Reduced_GDM_EE)
        print(Reduced_GDM_EE)

    # Plot the entanglement entropy
    #plt.figure(figsize=(10, 6))
    #plt.plot(range(nQubits+1), Reduced_GDM_EE_List, marker='o', linestyle='-', color='b')
    #plt.xlabel('Subsystem Index')
    #plt.ylabel('Entanglement Entropy')
    #plt.title('Entanglement Entropy for Each Subsystem')
    #plt.grid(True)
    #plt.show()

    return Reduced_GDM_EE_List
    

In [14]:
Random_State_EE_Function(8, 0, 1)

Original Qobj:
 Quantum object: dims=[[256], [256]], shape=(256, 256), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 0.00444738  0.          0.         ...  0.00109714 -0.00287744
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [ 0.00109714  0.          0.         ...  0.00027066 -0.00070985
   0.        ]
 [-0.00287744  0.          0.         ... -0.00070985  0.0018617
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]
256
8
[2, 2, 2, 2, 2, 2, 2, 2]
Reshaped Qobj:
 Quantum object: dims=[[2, 2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 2, 2]], shape=(256, 256), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 0.00444738  0.          0.         ...  0.00109714 -0.00287744
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.     

[(0.6794404915019583-0j),
 (0.6881503067115051-0j),
 (0.6794820374248691-0j),
 (0.6469446698359227-0j),
 (0.6686107654846507-0j),
 (0.6703485588868374-0j),
 (0.6857454288482692-0j),
 (0.6910444167886611-0j)]

I dont think the ptrace function works exactly as the one Junggi and Vinay explained. From the documentation it is explained as,

`ptrace acts on the Qobj instance for which it is called, and it takes one argument sel, which is a
list of integers that mark the component systems that should be kept. All other components are traced out.
`

Which sounds like its a different process than the one we want. We want to split the system into two subsystem, of A and B. Suppose that the system will be divided into 2 with the size of $m$ and $k$ for subsystem A and B, susbsystem A will start from $m = 1$ and subsystem B will then start from the size of $k = n-1$ 

In [15]:
array_testing_split = np.arange(9.0 + 1)
print(array_testing_split)
first, rest = array_testing_split[0], array_testing_split[1:]
print(first, rest)
first, rest = array_testing_split[:2], array_testing_split[2:]
print(first, rest)

[0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
0.0 [1. 2. 3. 4. 5. 6. 7. 8. 9.]
[0. 1.] [2. 3. 4. 5. 6. 7. 8. 9.]


In [16]:
psi = qt.tensor(qt.basis(2,0), qt.basis(2,1))
psi.ptrace(0)

Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[1. 0.]
 [0. 0.]]

In [17]:
psi.ptrace(1)

Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0. 0.]
 [0. 1.]]

This code is from https://github.com/qutip/qutip/issues/1076

In [18]:
def ptracealt(rho,qkeep) :
    rd = rho.dims[0]
    print(f"rd :",rd)
    nd = len(rd)
    print(f"nd :",nd)
    qkeep = list(np.sort(qkeep))
    print(f"qkeep :", qkeep)
    dkeep = (np.array(rd)[qkeep]).tolist()
    print(f"dkeep :",dkeep)
    qtrace = list(set(np.arange(nd))-set(qkeep))
    print(f"qtrace :",qtrace)
    dtrace = (np.array(rd)[qtrace]).tolist()
    print(f"dtrace :",dtrace)
    if qt.isket(rho) : # This part isnt really needed if we immediately uses dense matrix and not a ket. But its nice that the original coder
                        # put this here
        vmat = (rho.full()
                .reshape(rd)
                .transpose(qkeep+qtrace)
                .reshape([np.prod(dkeep),np.prod(dtrace)]))
        rhomat = vmat.dot(vmat.conj().T)
    else :
        rhomat = np.trace(rho.full() # Changes the Qobj form into an array of matrix.
                          .reshape(rd+rd) # Reshaping the Qobj into (rd + rd)
                          .transpose(qtrace+[nd+q for q in qtrace]+qkeep+[nd+q for q in qkeep])
                          # The transpose step is quite complicated, so we'll try to explain it in the markdown below.
                          .reshape([np.prod(dtrace),np.prod(dtrace),np.prod(dkeep),np.prod(dkeep)]))
    return qt.Qobj(rhomat,dims=[dkeep, dkeep])

```
rhomat = np.trace(rho.full() # Changes the Qobj form into an array of matrix.
                          .reshape(rd+rd) 
                          .transpose(qtrace+[nd+q for q in qtrace]+qkeep+[nd+q for q in qkeep])
                          .reshape([np.prod(dtrace),np.prod(dtrace),np.prod(dkeep),np.prod(dkeep)]))
```
before that, lets see what the function does before performing the partial trace,
```
    rd = rho.dims[0]
    nd = len(rd)
    qkeep = list(np.sort(qkeep))
    dkeep = (np.array(rd)[qkeep]).tolist()
    qtrace = list(set(np.arange(nd))-set(qkeep))
    dtrace = (np.array(rd)[qtrace]).tolist()
```
1. __Extract Dimensions of the Quantum State__

    `rd = rho.dims[0]`

    `rho.dims[0]`: This extracts the dimensions of the subsystems from the rho object.
    
    `rd` becomes a list of integers where each integer represents the dimension of a corresponding subsystem in the quantum state.
    
    For example, if the quantum state consists of three subsystems with dimensions 2, 3, and 4, then `rd = [2, 3, 4]`.

2. __Determine the Number of Subsystem__
    
    `nd = len(rd)`

    `nd` is the number of subsystem in the quantum state
    
    which is simply the length of the `rd` list

    from the previous example, it would have a value of `nd = 3`

3. __Sorting the subsystem to keep__

    `qkeep = list(np.sort(qkeep))`

    `qkeep`  is the list of indices of the subsystems that we want to keep after taking the partial trace.

    `np.sort(qkeep)`: Sorts the `qkeep` list in ascending order.

    `list(np.sort(qkeep))`: Converts the sorted array back to a list.

    If `qkeep = [2, 0]`, then after sorting, `qkeep = [0, 2]`.

4. __Get Dimensions of the Subsystems to Keep__

    `dkeep = (np.array(rd)[qkeep]).tolist()`

    `np.array(rd)`: Converts the `rd` list to a NumPy array for easier indexing.

    `np.array(rd)[qkeep]`: Uses the sorted qkeep indices to get the dimensions of the subsystems to keep.
    
    `tolist()`: Converts the result back to a list.
    
    For example, if `rd = [2, 3, 4]` and `qkeep = [0, 2]`, then `dkeep = [2, 4]`.

5. __Determine Subsystems to Trace Out__

    `qtrace = list(set(np.arange(nd)) - set(qkeep))`

    `np.arange(nd)`: Creates a NumPy array of integers from 0 to nd-1, representing all subsystem indices.

    `set(np.arange(nd))`: Converts this array to a set of all subsystem indices.

    `set(qkeep)`: Converts the `qkeep` list to a set.

    `set(np.arange(nd)) - set(qkeep)`: Computes the set difference, i.e., the indices of subsystems that are not in qkeep and hence should be traced out.
    
    `list(...)`: Converts the resulting set back to a list.
    
    Continuing the previous example, `np.arange(3) = [0, 1, 2]` and `qkeep = [0, 2]`, so `qtrace = [1]`.

6. __Get Dimensions of the Subsystems to Trace Out__

    `dtrace = (np.array(rd)[qtrace]).tolist()`

    `np.array(rd)`: Converts the `rd` list to a NumPy array.

    `np.array(rd)[qtrace]`: Uses the qtrace indices to get the dimensions of the subsystems to trace out.

    `tolist()`: Converts the result back to a list.

    For example, if `rd = [2, 3, 4]` and `qtrace = [1]`, then `dtrace = [3]`.

Example Walkthrough
Let's go through a concrete example to illustrate these steps.

Suppose:

The quantum system has three subsystems with dimensions __*rd = [d1, d2, d3]*__.

We want to keep the first subsystem __*(qkeep = [0])*__ and trace out the others __*(qtrace = [1, 2])*__.

- Initial Setup:
The full dimension list __*rd + rd*__ will be __*[d1, d2, d3, d1, d2, d3]*__.

- Transpose Order:
__*qtrace + [nd + q for q in qtrace]*__ becomes [1, 2, 4, 5].

__*qkeep + [nd + q for q in qkeep]*__ becomes [0, 3].

The transpose order is __*[1, 2, 4, 5, 0, 3]*__.

- Reshape After Transpose:
The new shape after transposing will be __*[d2, d3, d2, d3, d1, d1]*__.

This is then reshaped to __*[d2*d3, d2*d3, d1, d1]*__.

- Partial Trace:

__*np.trace(..., axis1=0, axis2=1)*__ performs the trace over the first two dimensions __*[d2*d3, d2*d3]*__, reducing the array to __*[d1, d1]*__, the density matrix of the subsystem we kept.

Finally, the function returns this reduced density matrix as a QuTiP quantum object with the appropriate dimensions for the kept subsystems.

In [19]:
rho_array = np.array([[0, 0, 0, 0, 1, 0, 1, 0], 
                      [0, 1, 0, 0, 0, 0, 0, 0], 
                      [0, 0, 0, 0, 0, 1, 0, 1], 
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 1, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 1],])
rho_array = qt.Qobj(rho_array)
rho_array.dims = [[2, 2, 2], [2, 2, 2]]

In [20]:
rho_array_ptrace_test = ptracealt(rho_array, [0,1])
rho_array_ptrace_test

rd : [2, 2, 2]
nd : 3
qkeep : [0, 1]
dkeep : [2, 2]
qtrace : [2]
dtrace : [2]


Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dense, isherm=False
Qobj data =
[[1. 0. 1. 1.]
 [0. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]

TESTING THE NEW PTRACEALT WITH LISTS OF THE SUBSYSTEM TO STAY, WHILE THE OTHER SUBSYSTEM TO BE TRACED OUT

In [64]:
print(len(test_density_matrix))
print(np.linalg.eig(test_density_matrix))
print(len(np.linalg.eig(test_density_matrix)[1]))

256
EigResult(eigenvalues=array([ 1.00000000e+00+0.00000000e+00j,  1.43099280e-17+0.00000000e+00j,
        1.32687923e-17+0.00000000e+00j, -1.13351990e-17+0.00000000e+00j,
       -1.12943466e-17+0.00000000e+00j, -1.08316148e-17+0.00000000e+00j,
        9.96904666e-18+0.00000000e+00j,  9.19901043e-18+0.00000000e+00j,
       -9.87276441e-18+0.00000000e+00j, -8.89639233e-18+0.00000000e+00j,
       -8.66403017e-18+0.00000000e+00j,  7.95330637e-18+1.69946059e-18j,
        7.95330637e-18-1.69946059e-18j,  7.92829134e-18+0.00000000e+00j,
       -7.10364956e-18+0.00000000e+00j,  7.03158048e-18+0.00000000e+00j,
       -7.21461558e-18+0.00000000e+00j, -6.49440589e-18+0.00000000e+00j,
        6.22743673e-18+0.00000000e+00j, -6.22877428e-18+0.00000000e+00j,
        5.68509163e-18+0.00000000e+00j,  5.87989506e-18+0.00000000e+00j,
       -5.60942851e-18+0.00000000e+00j, -5.09606466e-18+0.00000000e+00j,
       -4.93487311e-18+0.00000000e+00j,  5.24560285e-18+0.00000000e+00j,
        4.99790247e-18+0.

In [56]:
#test_density_matrix_ptracealt = np.linalg.eig(test_density_matrix)
test_density_matrix_ptracealt = qt.Qobj(test_density_matrix)
test_density_matrix_ptracealt.dims = [[2, 2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 2, 2]]
test_density_matrix_ptracealt = ptracealt(test_density_matrix_ptracealt, [0, 1, 2, 3, 4, 5, 6])
print(test_density_matrix_ptracealt)
print(np.trace(test_density_matrix_ptracealt.full()))

rd : [2, 2, 2, 2, 2, 2, 2, 2]
nd : 8
qkeep : [0, 1, 2, 3, 4, 5, 6]
dkeep : [2, 2, 2, 2, 2, 2, 2]
qtrace : [7]
dtrace : [2]
Quantum object: dims=[[2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 2]], shape=(128, 128), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 2.79425670e-03  9.46438443e-03  3.73348123e-03 ...  1.78706298e-03
  -2.86166444e-05 -6.47452023e-05]
 [ 9.46438443e-03  3.20604407e-02  1.26524307e-02 ...  6.00708618e-03
   0.00000000e+00  0.00000000e+00]
 [ 3.73348123e-03  1.26524307e-02  5.00072361e-03 ...  2.30484249e-03
   1.37024569e-04  3.10018299e-04]
 ...
 [ 1.78706298e-03  6.00708618e-03  2.30484249e-03 ...  1.70077349e-03
  -1.19767751e-03 -2.70974723e-03]
 [-2.86166444e-05  0.00000000e+00  1.37024569e-04 ... -1.19767751e-03
   2.49362046e-03  5.64182014e-03]
 [-6.47452023e-05  0.00000000e+00  3.10018299e-04 ... -2.70974723e-03
   5.64182014e-03  1.27646268e-02]]
(1+0j)


# EE Calculation from Scratch

Here's another (correct) way to calculate the EE,

First we will make a system where we describe the random variable states from a value of $[0, 1]$

$$
\ket{\psi} = \begin{pmatrix} 0.38 \\ 0.45 \\ 0.56 \\ \vdots \end{pmatrix}
$$

where the length of the column matrix is $N = 2^{n}$, with $n$ as the number of qubits we want to study and calculate the entanglement entropy.
Now that we have a random statevector, we would like to re-write it as a combination of its basis, 

$$
\ket{\psi} = \begin{pmatrix} 0.38 \\ 0.45 \\ 0.56 \\ \vdots \end{pmatrix} = 0.38 \begin{pmatrix} 1 \\ 0 \\ 0 \\ \vdots \end{pmatrix} + 0.45 \begin{pmatrix} 0 \\ 1 \\ 0 \\ \vdots \end{pmatrix} + 0.56 \begin{pmatrix} 0 \\ 0 \\ 1 \\ \vdots \end{pmatrix} + \cdots.
$$

We would need to normalize the statevector by taking $\rho = \sum_i a_i a^*_i\ket{i}\bra{i} = 1$.
Lets realize this into python,

In [22]:
def psi_initial_random_state(nQubits = 8, min_val = 0, max_val = 1):
    """
    Generate a row vector of random zeros and ones, and then each of the components are multiplied by a constant taken from a Gaussian ensemble
    for each of the components of the statevector.

    Parameters:
    nQubits (int): The length of the vector which corresponds to how many qubits.

    Returns:
    numpy.ndarray: A row vector of random numbers from [0, 1].
    """
    if not isinstance(nQubits, int) or nQubits <= 0:
        raise ValueError("The number of qubits must be a positive integer.")
    gaussian_ensemble = np.random.uniform(min_val, max_val, size = (nQubits, 1))
    #print(f"The gaussian ensemble for each of the components: \n", gaussian_ensemble)
    return gaussian_ensemble 

In [23]:
test_psi_initial_random_state = psi_initial_random_state(4)
test_psi_initial_random_state

array([[0.26709936],
       [0.39991554],
       [0.82016734],
       [0.52639184]])

now that we have a random state $\ket{\psi}$, we want to normalize the state so that $\rho = \sum_{ij} a_i a^*_j \ket{i}\bra{j} =1 $.

In [24]:
def normalize(matrix):
    norm = np.linalg.norm(matrix)
    matrix = matrix/norm  # normalized matrix
    return matrix
    return norm_arr

In [25]:
normalized_test_psi_initial_random_state = normalize(test_psi_initial_random_state)
print(f"Here's the normalized random state :\n",normalized_test_psi_initial_random_state)
Y = normalized_test_psi_initial_random_state*normalized_test_psi_initial_random_state
print(f"and here's the sum of the normalized density matrix :\n",sum(Y))

Here's the normalized random state :
 [[0.24577689]
 [0.36799038]
 [0.75469358]
 [0.48437011]]
and here's the sum of the normalized density matrix :
 [1.]


Okay, now that we have a normalized statevector, we can start to translate each of the components of the matrix into a binary number which corresponds to the state of the system.

For example, for an nQubit column vector, we have the first component symbolized as `psi[0]` with a certain normalized constant of $a_{[0]}$, as in python, the first component have the index of 0. This would represent the following state,
$$
\psi[0] \rightarrow a_{[0]}\ket{00000\cdots 00},
$$
where the length of the ket is nQubits.

and for $\psi_{[1]}$, it would correspond a state of,
$$
\psi[1] \rightarrow a_{[1]}\ket{00000\cdots 01},
$$
and so on. Here it is clear that our system just use the index of the array and translate it into binary that will correspond into the actual quantum state.
Before thinking about how to separate the system into 2 sub-system, lets translate it first.

From Vinay's explanation, the first step I want to do is to determine the size of subsystem A and B as $n_A$ and $n_B$ respectively.

So for a system with a total of 4 qubits, I want to start from $n_A = 1$ and $n_B = 3$, where I would need to create a basis with the size of $2^{n_A}$ and $2^{n_B}$.


In [128]:
n_A = 1
n_B = 3

Creating the basis

In [129]:
basis_A = np.random.uniform(0, 1, (2**n_A, 1))
print(f"Basis for subsystem A :\n",basis_A)
basis_B = np.random.uniform(0, 1, (2**n_B, 1))
print(f"Basis for subsystem B :\n",basis_B)

Basis for subsystem A :
 [[0.30942363]
 [0.73618177]]
Basis for subsystem B :
 [[0.27342197]
 [0.04149395]
 [0.09536258]
 [0.40783826]
 [0.59481355]
 [0.41369378]
 [0.42140203]
 [0.35512608]]


In [130]:
Psi_AB = np.kron(basis_A, basis_B)
print(f"State of 'Psi' as kronecker product of basis A and B :\n",
      Psi_AB)

State of 'Psi' as kronecker product of basis A and B :
 [[0.08460322]
 [0.01283921]
 [0.02950744]
 [0.1261948 ]
 [0.18404937]
 [0.12800663]
 [0.13039174]
 [0.1098844 ]
 [0.20128827]
 [0.03054709]
 [0.0702042 ]
 [0.30024309]
 [0.43789089]
 [0.30455382]
 [0.31022849]
 [0.26143735]]


In [131]:
len(Psi_AB)

16

Normalizing the column vector (State of $\Psi$)

In [132]:
norm = np.linalg.norm(Psi_AB)

# Normalize the vector
normalized_Psi_AB = Psi_AB / norm
print(f"The normalized state of Psi :\n", normalized_Psi_AB)

The normalized state of Psi :
 [[0.10191886]
 [0.01546699]
 [0.03554669]
 [0.15202293]
 [0.22171853]
 [0.15420559]
 [0.15707887]
 [0.13237431]
 [0.2424857 ]
 [0.03679912]
 [0.0845728 ]
 [0.36169348]
 [0.52751349]
 [0.36688648]
 [0.37372258]
 [0.31494541]]


In [176]:
def Psi_indexed(i, j):
    if isinstance(i, int) and i > 0 and i <= len(basis_A) and isinstance(j, int) and j > 0 and j <= len(basis_B):
        return normalized_Psi_AB[(i-1) * len(basis_B) + (j-1)]
    else:
        return ValueError("Indices are out of bounds or not integers.")

In [146]:
Psi_indexed(i = 2,j = 1)

array([0.2424857])

In [150]:
def Psi_indexed_conjugate(k, l):
    if isinstance(k, int) and k > 0 and k <= len(basis_A) and isinstance(l, int) and l > 0 and l <= len(basis_B):
        return np.conjugate(normalized_Psi_AB[(k-1) * len(basis_B) + (l-1)])
    

In [151]:
def Density_Matrix_Psi_AB(i, j, k, l):
    # Summing over Psi(i,j) and Psi*(k, j), sum over J from 1 to the len(basis_B).
    Density_Matrix_Psi_AB = Psi_indexed(i, j) * Psi_indexed_conjugate(k, l)
    return Density_Matrix_Psi_AB

In [256]:
def Partial_Trace_Subsystem_B(i, k):
    result = 0
    for j in range(len(basis_B)):
        result += Density_Matrix_Psi_AB(i, (j+1), k, (j+1))
    return result[0]
        

In [248]:
range(len(basis_A) -1)

range(0, 1)

In [270]:
Partial_Trace_Subsystem_B(1, 1)

0.15013641935753763

In [275]:
array_test_eigvalue = np.array(((Partial_Trace_Subsystem_B(1, 1), Partial_Trace_Subsystem_B(1, 2))),
                               ((Partial_Trace_Subsystem_B(2, 1), Partial_Trace_Subsystem_B(2, 2))))
print(array_test_eigvalue)

[0.15013642 0.35720509]


In [269]:
def Partial_Trace_Subsystem_B_array_test():
    matrix = np.zeros((len(basis_A), len(basis_A)))
    
    for i in range(len(basis_A) - 1):
        for k in range(len(basis_A) - 1):
            matrix[i, k] = Partial_Trace_Subsystem_B((i + 1), (k + 1))
    return print(matrix)

In [264]:
#print(Partial_Trace_Subsystem_B_array_test())
eig_test = Partial_Trace_Subsystem_B_array_test()

[[0.15013642 0.        ]
 [0.         0.        ]]


In [224]:
matrix = np.array([[Partial_Trace_Subsystem_B(i+1, j+1) for j in range(len(basis_A))] for i in range(len(basis_A))])
print(matrix)
print(matrix[0])

[[[0.15013642]
  [0.35720509]]

 [[0.35720509]
  [0.84986358]]]
[[0.15013642]
 [0.35720509]]


In [263]:
eigenvalues, eigenvectors = np.linalg.eig(eig_test)
print("Eigenvalues of the partial trace matrix:")
print(eigenvalues)

LinAlgError: 0-dimensional array given. Array must be at least two-dimensional

In [235]:
testmatrix = ((0, 1), (3, 1))
#print(testmatrix)

row_vector = np.array((1, 2))
#print(row_vector)

vector = np.array(((1, 2), (3, 4)))
print(vector)

eigenvalue_test, eigen_bases = np.linalg.eig(vector)
print(eigenvalue_test)

[[1 2]
 [3 4]]
[-0.37228132  5.37228132]


In [3]:
import numpy as np

## EE Calculation version 3 (?)

In [4]:
def psi_initial_random_state_gaussian(nQubits = 8, mean = 0, std = 1):
    """
    Generate a row vector of random zeros and ones, and then each of the components are multiplied by a constant taken from a Gaussian ensemble
    for each of the components of the statevector.

    Parameters:
    nQubits (int): The length of the vector which corresponds to how many qubits.

    Returns:
    numpy.ndarray: A row vector of random numbers from [0, 1].
    """
    if not isinstance(nQubits, int) or nQubits <= 0:
        raise ValueError("The number of qubits must be a positive integer.")
    gaussian_ensemble = np.random.normal(mean, std, size = (2**nQubits, 1))
    #print(f"The gaussian ensemble for each of the components: \n", gaussian_ensemble)
    normalized_gaussian_ensemble = gaussian_ensemble / np.linalg.norm(gaussian_ensemble)
    return normalized_gaussian_ensemble

In [5]:
gaussian_statevector = psi_initial_random_state_gaussian(4, 0, 1)
print(gaussian_statevector)

[[-0.15580019]
 [-0.14095701]
 [-0.53670439]
 [ 0.06486537]
 [-0.29441512]
 [-0.22848133]
 [ 0.28330763]
 [-0.13275483]
 [-0.20026937]
 [ 0.04641477]
 [ 0.07932171]
 [-0.12728571]
 [ 0.51720051]
 [ 0.22172062]
 [ 0.15771338]
 [-0.14332346]]


In [6]:
def psi_initial_random_state_uniform(nQubits = 8, min_val = 0, max_val = 1):
    """
    Generate a row vector of random zeros and ones, and then each of the components are multiplied by a constant taken from a Gaussian ensemble
    for each of the components of the statevector.

    Parameters:
    nQubits (int): The length of the vector which corresponds to how many qubits.

    Returns:
    numpy.ndarray: A row vector of random numbers from [0, 1].
    """
    if not isinstance(nQubits, int) or nQubits <= 0:
        raise ValueError("The number of qubits must be a positive integer.")
    uniform_ensemble = np.random.uniform(min_val, max_val, size = (2**nQubits, 1))
    normalized_uniform_ensemble = uniform_ensemble / np.linalg.norm(uniform_ensemble)
    #print(f"The gaussian ensemble for each of the components: \n", gaussian_ensemble)
    return normalized_uniform_ensemble

In [7]:
uniform_statevector = psi_initial_random_state_uniform(4, 0, 1)
print(uniform_statevector)

[[0.39776774]
 [0.14161104]
 [0.29537197]
 [0.241988  ]
 [0.25556374]
 [0.29273189]
 [0.37443201]
 [0.09745124]
 [0.37727287]
 [0.37517653]
 [0.08428839]
 [0.06028221]
 [0.16201877]
 [0.00214536]
 [0.18051341]
 [0.1501765 ]]


In [8]:
np.log2(len(uniform_statevector))

4.0

Next is that we have the Psi_indexed function that can take specific qubits which will corresponds to the divided subsystems

In [9]:
def Psi_indexed_v2(normalized_matrix, n_A, i, j):
    n_B = int(np.log2(len(normalized_matrix)) - n_A)
    if isinstance(i, int) and i > 0 and i <= 2**n_A and isinstance(j, int) and j > 0 and j <= 2**n_B:
        return normalized_matrix[(i - 1) * 2**n_B + (j - 1)]
    else:
        return ValueError("Indices are out of bounds or not integers")

In [10]:
def Psi_indexed_conjugate_v2(normalized_matrix, n_A, k, l):
    n_B = int(np.log2(len(normalized_matrix)) - n_A)
    if isinstance(k, int) and k > 0 and k <= 2**n_A and isinstance(l, int) and l > 0 and l <= 2**n_B:
        return np.conjugate(normalized_matrix[(k - 1) * 2**n_B + (l - 1)])
    else:
        return ValueError("Indices are out of bounds or not integers")

In [11]:
test_gaussian_psi_indexed = Psi_indexed_v2(gaussian_statevector, 2, 1, 1)
print(test_gaussian_psi_indexed)
test_uniform_psi_indexed = Psi_indexed_v2(uniform_statevector, 2, 1, 1)
print(test_uniform_psi_indexed)

[-0.15580019]
[0.39776774]


In [12]:
test_gaussian_psi_indexed = Psi_indexed_conjugate_v2(gaussian_statevector, 2, 1, 1)
print(test_gaussian_psi_indexed)
test_uniform_psi_indexed = Psi_indexed_conjugate_v2(uniform_statevector, 2, 1, 1)
print(test_uniform_psi_indexed)

[-0.15580019]
[0.39776774]
